# Preprocessing Data 

In [7]:
# setup
import os, sys
'''
LOCAL and COLLAB, ONLY one of the two environements should be set True.
'''
# running on local desktop
LOCAL = True

# running on UVA collab
UVA_COLAB = False

#print current directory
print(f"current directory: {os.getcwd()}")

current directory: C:\Users\mhomb\Documents\Sandbox\myprojects\case-expungement


In [8]:
# setup environment variables
%run -i setup.py

In [9]:
# import all required libraries
%run -i settings.py

#import all utility functions
%run -i utility-functions.py

Imports and display options set...
Loading all utility functions...


In [10]:
# To auto-reload modules in jupyter notebook (so that changes in files *.py doesn't require manual reloading):
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%reload_ext autoreload
%autoreload 2

# you can now enable 2x images by just adding the line:
# see: https://gist.github.com/minrk/3301035
%config InlineBackend.figure_format = 'png'

In [11]:
import bamboolib as bam
file = f'{BASE_DIR}/data/circuit_court_2009_2019.csv.gz'
df = pd.read_csv(file, parse_dates =['offense_date'])
df.head(5)

,person_id,offense_date,final_disposition,fips,gender,race,charge,charge_type,ammended_charge
0,152051000000121,2007-01-09,Nolle Prosequi,99,Female,White Caucasian (Non-Hispanic),FAILURE TO APPEAR,Felony,NaN
1,152051000000121,2007-01-09,Nolle Prosequi,99,Female,White Caucasian (Non-Hispanic),FAILURE TO APPEAR,Felony,NaN
2,152051000000121,2007-01-09,Nolle Prosequi,99,Female,White Caucasian (Non-Hispanic),FAILURE TO APPEAR,Felony,NaN
3,152051000000121,2007-01-09,Nolle Prosequi,99,Female,White Caucasian (Non-Hispanic),FAILURE TO APPEAR,Felony,NaN
4,152051000000121,2006-12-13,Sentence/Probation Revoked,99,Female,White Caucasian (Non-Hispanic),FAIL COMPLY W/ COND SUSP SENT,Felony,NaN


### Summary Statistics

In [12]:
# In this example, the data frame is described and [‘object’] is passed to include parameter
# to see description of object series. [.20, .40, .60, .80] is passed to percentile parameter 
# to view the respective percentile of Numeric series.
# see: https://www.geeksforgeeks.org/python-pandas-dataframe-describe-method/
perc = [0.20, .40, .60, 0.80]
include = ['object', 'float', 'int']
df.describe(percentiles= perc, include=include).T

,count,unique,top,freq
offense_date,1990315,11192,2013-03-26,1624
final_disposition,1933598,12,Guilty,873527
gender,1990315,2,Male,1496894
race,1987177,10,White Caucasian (Non-Hispanic),987196
charge,1990311,201508,VIOL PROBATION ON FEL OFF,137303
charge_type,1990315,6,Felony,1388595
ammended_charge,167943,19377,PETIT LARCENY,7739


## Data Cleaning

In [33]:
# number of na values in each column
df.isna().sum()

person_id                  0
offense_date               0
final_disposition      56717
fips                       0
gender                     0
race                    3138
charge                     4
charge_type                0
ammended_charge      1822372
dtype: int64

For our study, 'personId', 'offense_date', 'final_disposition', 'fips', 'race', 'gender', 'charge', 'fips_area' are mandatory columns.
So removing all records where these field values are missing will be removed

In [34]:
df.dropna(axis=0, subset=['final_disposition'], inplace=True)
df.dropna(axis=0, subset=['race'], inplace=True)

In [35]:
df.isna().sum()

person_id                  0
offense_date               0
final_disposition          0
fips                       0
gender                     0
race                       0
charge                     0
charge_type                0
ammended_charge      1763917
dtype: int64

In [36]:
# for the moment we consider case_class as important field, so we will impute missing value 'unknown'
df['ammended_charge'].fillna('unknown', inplace=True)

In [37]:
df.isna().sum()

person_id            0
offense_date         0
final_disposition    0
fips                 0
gender               0
race                 0
charge               0
charge_type          0
ammended_charge      0
dtype: int64

In [38]:
duplicate_records = pd.DataFrame(df.duplicated(), columns=['isduplicate'])
duplicate_records = duplicate_records.reset_index()
duplicate_records.columns = [str(column) for column in duplicate_records.columns]
duplicate_records.set_index('index')
duplicate_records.groupby('isduplicate').count()

,index
isduplicate,
False,1646690
True,284323


Our dataset has 974935 duplicate records, so we purge these records from our dataset

In [26]:
#drop duplicate records
df = df.drop_duplicates(['person_id', 'offense_date', 'final_disposition', 'fips', 'race', 'gender', 'charge', 'ammended_charge'])
len(df)

1638097

df

* We can see that hearing_date, fips, gender, charge_type and person_id do not have any missing data, however we need to address the missing data  for other columns.



In [ ]:
df.isna().sum()

Per our domain expert if ammended_charge is reduced to **'DWI'** then the case is a candidate for expungement.

In [ ]:
dwi_ammended_charge_df =  df [df.ammended_charge.str.contains('DWI') == True]
len(dwi_ammended_charge_df)

Our aim is to derive the cadidate cases for expungement based on hearing_results, let us examine the uniuqe values for hearing_result

In [ ]:
# top 15 hearing results
hearing_result_counts = df['final_disposition'].value_counts()
subset = hearing_result_counts[:15]
sns.barplot(y=subset.index, x=subset.values)

In [ ]:
# to 15 hearing results by gender
df_result_bygendger = df.groupby(['final_disposition', 'gender'])\
                        .size()\
                        .unstack()\
                        .fillna(0)\
                        .sort_values(['Female', 'Male'], ascending=False)
df_stacked = df_result_bygendger.head(15).stack()
df_stacked.name = 'total'
df_stacked= df_stacked.reset_index()
sns.barplot(x='total', y='final_disposition', hue='gender', data= df_stacked)

## Preprocessing Data

### Adding fips_area name column

In [ ]:
#load fips code table
fips_file = 'reference-data/va-fips-codes.csv'
fips_df = pd.read_csv(fips_file)
fips_df = fips_df[['CountyFIPSCode', 'GUName']]
fips_df

In [ ]:
#add fips_GUName
df = pd.merge(df,fips_df,left_on='fips', right_on='CountyFIPSCode',  how='left')\
    .drop(columns=['CountyFIPSCode'], axis=1)\
    .rename(columns={'GUName': 'fips_area'})
df

## Identifying the candidates for case expungement

Assumptions maded based on domain expert input:
1. If the final_disposition is any the following values 'Dismissed','Noile Prosequi','Not Guilty', 'Withdrawn', 'Not Found', 'No Indictment Presented', 'No Longer Under Advisement', 'Not True Bill' then the case is candidate for expungement.
2. **TODO:** If the charges are ammended to DWI then also it can be a candidate for expungement if the person does not have any prior felony charges 


In [ ]:
cand_list =['Dismissed','Noile Prosequi','Not Guilty', 'Withdrawn', 'Not Found', 'No Indictment Presented', 'No Longer Under Advisement', 'Not True Bill']

In [ ]:
df['candidate'] = [1 if x in cand_list else 0 for x in df['final_disposition']]

In [14]:
df

,person_id,offense_date,final_disposition,fips,gender,race,charge,charge_type,ammended_charge
0,152051000000121,2007-01-09,Nolle Prosequi,99,Female,White Caucasian (Non-Hispanic),FAILURE TO APPEAR,Felony,NaN
1,152051000000121,2007-01-09,Nolle Prosequi,99,Female,White Caucasian (Non-Hispanic),FAILURE TO APPEAR,Felony,NaN
2,152051000000121,2007-01-09,Nolle Prosequi,99,Female,White Caucasian (Non-Hispanic),FAILURE TO APPEAR,Felony,NaN
3,152051000000121,2007-01-09,Nolle Prosequi,99,Female,White Caucasian (Non-Hispanic),FAILURE TO APPEAR,Felony,NaN
4,152051000000121,2006-12-13,Sentence/Probation Revoked,99,Female,White Caucasian (Non-Hispanic),FAIL COMPLY W/ COND SUSP SENT,Felony,NaN
...,...,...,...,...,...,...,...,...,...
1990310,30150000000657,2018-04-16,Guilty,89,Male,White,B&E DWELL W/ARMED DEADLY WPN,Felony,ENT W/INT INTERFERE
1990311,30150000000657,2018-04-16,Guilty,89,Male,White,A&B FAM/HH MEMB,Misdemeanor,NaN
1990312,347070000000282,2018-11-30,Dismissed,89,Male,Black,BOND APPEAL,"Other (Animal Violations, Bond Appeals)",NaN
1990313,131071000000437,2018-11-10,Guilty,89,Female,White,SHOPLIFT 3RD/SUB,Felony,NaN


In [ ]:
df.groupby(['candidate']).count().head()

In [ ]:
df.groupby(['candidate','race','gender']).count()

In [ ]:
df.to_csv(
    PROCESSED_PATH + "district_court_2009_2019_cleansed.csv.gz",
    index=False,
    compression="gzip",
    header=True,
    quotechar='"',
    doublequote=True,
    line_terminator="\n",
)

In [ ]:
delete_file(PROCESSED_PATH + "district_court_2009_2019.csv.gz")

In [ ]:
#!jupyter nbconvert va_circuit_court_eda.ipynb --to pdf